In [8]:
#pip install pyemvue|

In [16]:
import pyemvue
from pyemvue.enums import Scale, Unit
from datetime import datetime, timezone, timedelta
import csv

In [10]:
vue = pyemvue.PyEmVue()
vue.login(username='xxxxxxxx@dominio.com', password='contraseña')

True

In [11]:
devices = vue.get_devices()
device_gids = []
device_info = {}
for device in devices:
    if not device.device_gid in device_gids:
        device_gids.append(device.device_gid)
        device_info[device.device_gid] = device
    else:
        device_info[device.device_gid].channels += device.channels

In [12]:
def estructurar_data(date_filter,usage_dict, info, depth=0):
        
    list_log = list()    
    for gid, device in usage_dict.items():
        for channelnum, channel in device.channels.items():
            name = channel.name
            if name == 'Main':
                name = info[gid].device_name
            
            dict_log = dict()
            dict_log['fecha'] = date_filter.strftime('%Y-%m-%d %H:%M:%S')
            dict_log['idDispositivo'] = gid
            dict_log['idCanal'] = channelnum
            dict_log['nombreCanal'] = name
            dict_log['usoCanal'] = channel.usage
            list_log.append(dict_log)
            
    return list_log            

In [13]:
inicio_del_dia = datetime(2024, 6, 21, 0, 0, tzinfo=timezone(timedelta(hours=-5)))
list_reporte = list()    

for i in range(0, 24):  
    tiempo = inicio_del_dia + timedelta(minutes=60 * i)
    device_usage_dict = vue.get_device_list_usage(deviceGids=device_gids, instant=tiempo, scale=Scale.HOUR.value, unit=Unit.KWH.value)
    list_respuesta=(estructurar_data(tiempo,device_usage_dict, device_info))
    list_reporte = list_reporte+list_respuesta     

In [15]:
list_reporte

[{'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '1,2,3',
  'nombreCanal': 'Casa #1',
  'usoCanal': 0.5025715148062175},
 {'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '1',
  'nombreCanal': 'Refrigeradora',
  'usoCanal': 0.07029546610063976},
 {'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '2',
  'nombreCanal': 'Tomacorrientes P/A',
  'usoCanal': 0.02347467891348733},
 {'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '3',
  'nombreCanal': 'Tomacorrientes P/B y lavadora',
  'usoCanal': 0.02015854882346259},
 {'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '4',
  'nombreCanal': 'A.A cuarto 1',
  'usoCanal': 0.18138710196548039},
 {'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '5',
  'nombreCanal': 'A. A cuarto 1',
  'usoCanal': 0.1808881651828024},
 {'fecha': '2024-06-21 00:00:00',
  'idDispositivo': 238407,
  'idCanal': '6',
  'nombreCa

In [18]:
if list_reporte:
    campos = list(list_reporte[0].keys())

    with open('Reporte_Consumo_Electrico_'+inicio_del_dia.strftime('%Y_%m_%d')+'_1H.csv', mode='w', newline='', encoding='utf-8') as archivo:
        escritor_csv = csv.DictWriter(archivo, fieldnames=campos, delimiter=';')

        escritor_csv.writeheader()

        for dato in list_reporte:
            escritor_csv.writerow(dato)

    print(f'Archivo creado con éxito.')
else:
    print('La lista de datos está vacía.')

Archivo creado con éxito.
